In [1]:
import math 
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Parametros algoritmo

In [2]:
Muta = 0.15 #Probabilidad de mutacion
N_padres = 40 #Numero de padres en la seleccion de cruces
N_Poblacion = 100 #Numero de individuos en la poblacion
GenMax = 500 #Número de generaciones max


In [3]:
Stream_Lit_Pres = 1500  #Resultado dado por Stream Lit, valor por defecto = 1500€

#Presupuesto del Arbol Genetico
#Restamos 120€ de fuente de alimentacion generica
#Restamos 70€ de caja generica
#Restamos 110# de placa base
Presupuesto = Stream_Lit_Pres - 120 - 70 - 110

Importacion de csv + info

In [4]:
CPU = pd.read_csv('cpu_ferran_clean.csv')
GPU = pd.read_csv('gpu_ferran_clean.csv')
SSD = pd.read_csv('SSD_limpio.csv')
RAM = pd.read_csv('Ram Total.csv')
PS = pd.read_csv('power-supply_clean.csv')
CASE = pd.read_csv('case_clean.csv')
MB = pd.read_csv('motherboard_clean.csv')

print(f'CPU tiene {len(CPU)} lineas, GPU tiene {len(GPU)} lineas, SSD tiene {len(SSD)} lineas y RAM tiene {len(RAM)} lineas,')

CPU tiene 95 lineas, GPU tiene 850 lineas, SSD tiene 48 lineas y RAM tiene 150 lineas,


In [5]:
Comp_total = len(CPU) + len(GPU) + len(SSD) + len(RAM)
print(f'Cantidad de componentes totales: {Comp_total}')

Cantidad de componentes totales: 1143


In [381]:
CPU.head(1)

,Nombre,Precio,Numero_Procesamiento,Frecuencia_Maxima,Calor_Max_Teorico,SMT,Ranking
0,Intel Core i9-14900KS,699.99,24,6.2,150,1,1


In [382]:
GPU.head(1)

,Fabricante,Precio,Chip,Memoria,Nucleo_Reloj,Impulsion_Reloj,Longitud,Ranking
0,MSI VENTUS 3X E OC,2149.98,GeForce RTX 4090,24,2235,2565,322,1


In [383]:
SSD.head(1)

,Nombre,Precio,Capacidad (GB),Interfaz,Fabricante,Rank
0,WD Black SN850X,279.99,4000,M.2 PCIe 4.0 X4,WD,1


In [384]:
RAM.head(1)

,Nombre,Latencia (ns),Lectura(GB/s),Write (GB/s),Capacidad,Modelo,Rank,Precio
0,G Skill Intl F5-6000J3040G32G,38,21.9,19.2,32,Ram ddr5,1,222


# Funciones

Funcion de obtencion de genes para un cromosoma aleatoriamente

In [385]:
def RandomCrom (Crom_List = [], Pres = Presupuesto, CPU = CPU, GPU = GPU, SSD = SSD, RAM = RAM):

    Crom_List = []

    Total = 0

    while Total > Pres or Total == 0:
        
        CPU_Crom = random.randint(0,len(CPU)-1)
        GPU_Crom = random.randint(0,len(GPU)-1)
        SSD_Crom = random.randint(0,len(SSD)-1)
        RAM_Crom = random.randint(0,len(RAM)-1)

        Total = CPU['Precio'][CPU_Crom] + GPU['Precio'][GPU_Crom] + SSD['Precio'][SSD_Crom] + RAM['Precio'][RAM_Crom]

    Crom_List.append(CPU_Crom)
    Crom_List.append(GPU_Crom)
    Crom_List.append(SSD_Crom)
    Crom_List.append(RAM_Crom)

    return Crom_List
    

Funcion de obtencion del ranking de cada gen dentro de un cromosoma

In [386]:
def RankCrom (crom_list : list, df_cpu = CPU, df_gpu = GPU, df_ssd = SSD, df_ram = RAM):

    Rank_List = []
    

    Rank_List.append(df_cpu['Ranking'][crom_list[0]])
    Rank_List.append(df_gpu['Ranking'][crom_list[1]])
    Rank_List.append(df_ssd['Rank'][crom_list[2]])
    Rank_List.append(df_ram['Rank'][crom_list[3]])

    return Rank_List

Funcion obtencion del fitness, en este caso la suma de los rankings (cuanto mas pequeño mejor)

In [387]:
def FitMaker(RankPob : list):

    Fitness_list = []

    for i in range(len(RankPob)):        

        Fitness_list.append(sum(RankPob[i]))

    return Fitness_list
    

Funcion obtencion de precio de cada cromosoma

In [388]:
def PrecioCrom(Pob : list):

    Precio_list = []

    for i in range(len(Pob)):

        Precio_cr = 0

        for j in range(len(Pob[i])):
            
            if j == 0:
                Precio_cr += CPU['Precio'][Pob[i][j]]
            if j == 1:
                Precio_cr += GPU['Precio'][Pob[i][j]]
            if j == 2:
                Precio_cr += SSD['Precio'][Pob[i][j]]
            if j == 3:
                Precio_cr += RAM['Precio'][Pob[i][j]]
        
        Precio_list.append(round(Precio_cr,2))

    return Precio_list

Funcion obtencion hijos nuevos

In [389]:
def CruceNCromosomas(n = N_padres, Pob = [], i_padres = [], m = Muta):

    padres = []
    hijos_totales = []
    precio_hijos = []
    Precio_1 = []
    Precio_2 = []

    random.shuffle(i_padres)

    for i in range(0, len(i_padres), 2):     # Primero creamos las parejas 

        padres.append([i_padres[i], i_padres[i+1]])        

    for j in range(len(padres)):    #Creacion de nuevos hijos mediante cruce homogeneo (intercambiamos)

        hijo_1 = []
        hijo_2 = []

        lista_aux = Pob[padres[j][0]]

        for k in range(len(Pob[padres[j][0]])):

            if k % 2 == 0:
                
                hijo_1.append(Pob[padres[j][0]][k])
                hijo_2.append(Pob[padres[j][1]][k])

            else:

                hijo_1.append(Pob[padres[j][1]][k])
                hijo_2.append(Pob[padres[j][0]][k])


        if random.random() < m: #Calculamos aleatoriamente si muta o no
            
            nrandom = random.randint(0,len(hijo_1)-1)
            Crom_random = RandomCrom()

            hijo_1[nrandom] = Crom_random[nrandom]
        
        if random.random() < m: #Calculamos aleatoriamente si muta o no
            
            nrandom = random.randint(0,len(hijo_2)-1)
            Crom_random = RandomCrom()

            hijo_2[nrandom] = Crom_random[nrandom]
        
        Precio_1 = []
        Precio_2 = []
        aux1 = []
        aux2 = []

        aux1.append(hijo_1)
        aux2.append(hijo_2)

        Precio_1 = PrecioCrom(aux1)
        Precio_2 = PrecioCrom(aux2)
        
        if Precio_1[0] < Presupuesto:
            hijos_totales.append(hijo_1)
            precio_hijos.append(Precio_1[0])


        if Precio_2[0] < Presupuesto:
           hijos_totales.append(hijo_2)
           precio_hijos.append(Precio_2[0])


    return hijos_totales

# Obtencion de poblacion inicial (N Cromosomas, Valor total < Presupuesto)

In [390]:
Poblacion = []
Rank_Poblacion = []
Precio_Poblacion = []

for i in range(N_Poblacion):

    Cromosoma = RandomCrom()
    Rank_Cromosoma = RankCrom(Cromosoma)

    Poblacion.append(Cromosoma)
    Rank_Poblacion.append(Rank_Cromosoma)

Precio = PrecioCrom(Poblacion)
Fitness = FitMaker(Rank_Poblacion)

print('Poblacion:')
print(Poblacion)
print('-'*200)

print('Ranking poblacion:')
print(Rank_Poblacion)
print('-'*200)

print('Fitness:')
print(Fitness)
print('-'*200)

print('Precio:')
print(Precio)
print('-'*200)

Poblacion:
[[33, 824, 3, 53], [78, 704, 44, 83], [82, 776, 26, 57], [32, 819, 18, 22], [48, 653, 17, 140], [76, 604, 43, 105], [37, 730, 16, 18], [79, 661, 3, 71], [77, 209, 44, 62], [84, 673, 45, 95], [86, 648, 35, 77], [22, 535, 15, 84], [36, 815, 31, 103], [43, 244, 21, 62], [42, 62, 43, 94], [10, 558, 46, 71], [12, 755, 33, 3], [25, 534, 8, 144], [65, 518, 43, 82], [49, 310, 44, 31], [35, 409, 1, 17], [78, 643, 44, 12], [6, 423, 41, 45], [22, 747, 16, 102], [36, 837, 4, 84], [68, 445, 8, 41], [70, 765, 13, 78], [67, 488, 12, 86], [86, 532, 11, 104], [75, 666, 14, 47], [46, 818, 13, 66], [19, 551, 18, 105], [55, 460, 31, 99], [32, 492, 27, 37], [69, 680, 12, 27], [77, 818, 7, 7], [73, 461, 33, 131], [62, 177, 46, 134], [48, 396, 42, 79], [71, 760, 10, 95], [13, 653, 13, 103], [84, 406, 11, 146], [21, 794, 20, 129], [92, 253, 27, 50], [90, 387, 14, 56], [4, 719, 5, 72], [68, 821, 33, 135], [68, 449, 5, 44], [88, 401, 36, 146], [35, 344, 41, 54], [26, 506, 9, 115], [73, 720, 47, 41], 

In [391]:
Fitness_inicial = FitMaker(Poblacion)
print(Fitness_inicial)

[913, 909, 941, 891, 858, 828, 801, 814, 392, 897, 846, 656, 985, 370, 241, 685, 803, 711, 708, 434, 462, 777, 515, 887, 961, 562, 926, 653, 733, 802, 943, 693, 645, 588, 788, 909, 698, 419, 565, 936, 782, 647, 964, 422, 547, 800, 1057, 566, 671, 474, 656, 881, 708, 841, 822, 950, 614, 978, 842, 667, 587, 429, 687, 981, 462, 516, 910, 800, 985, 846, 974, 574, 986, 763, 412, 588, 945, 1084, 783, 737, 1050, 970, 577, 771, 437, 511, 675, 426, 798, 547, 537, 754, 843, 909, 529, 960, 320, 788, 610, 621]


# Cruce de poblacion

In [392]:
for Gen in range(GenMax):
    # Obtenemos los N_padres valores más grandes junto con sus índices
    Cromosomas_Padre = []

    for posicion, valor in enumerate(Poblacion):

        if len(Cromosomas_Padre) < N_padres:    # Llenamos primero la lista con los N_padres primeros cromosomas

            Cromosomas_Padre.append((valor, posicion))

        else:
            
            maximo_actual, max_posicion = min(Cromosomas_Padre) # Si ya tenemos N_padres valores, comparamos con el mínimo actual

            if valor < maximo_actual:

                Cromosomas_Padre.remove((maximo_actual, max_posicion))
                Cromosomas_Padre.append((valor, posicion))

    indices_Cromosomas_Padre = [posicion for _, posicion in Cromosomas_Padre]   # Extraemos solo los índices de los valores más grandes

    print('Generacion:')
    print(Gen)
    print('-'*200)
    print('Poblacion:')
    print(Poblacion)
    print('-'*200)
    print(f"Índices de los {N_padres} con un Fitness mas alto:")
    print(indices_Cromosomas_Padre)

    Hijos = CruceNCromosomas(Pob = Poblacion, i_padres = indices_Cromosomas_Padre)

    print('-'*200)
    print('Hijos:')
    print(Hijos)
    print('-'*200)

    Poblacion_Nueva = []
    indices_Poblacion_Nueva = []

    Poblacion.extend(Hijos)

    for posicion, valor in enumerate(Poblacion):

        if len(Poblacion_Nueva) < N_Poblacion:    # Llenamos primero la lista con los N_Poblacion primeros individuos

            Poblacion_Nueva.append(valor)
            indices_Poblacion_Nueva.append(posicion)

        else:
            
            maximo_actual = min(Poblacion_Nueva) # Si ya tenemos N_Poblacion valores, comparamos con el mínimo actual
            max_posicion = indices_Poblacion_Nueva[Poblacion_Nueva.index(maximo_actual)]
        

            if valor < maximo_actual:

                Poblacion_Nueva.remove(maximo_actual)
                indices_Poblacion_Nueva.remove(max_posicion)
                Poblacion_Nueva.append(valor)
                indices_Poblacion_Nueva.append(posicion)

    Poblacion = Poblacion_Nueva
    print(f"Índices de los {N_Poblacion} cromosomas con un Fitness mas alto:")
    print(indices_Poblacion_Nueva)
    print('-'*200)

Fitness_Total = FitMaker(Poblacion)
print(Fitness_Total)

Generacion:
0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Poblacion:
[[33, 824, 3, 53], [78, 704, 44, 83], [82, 776, 26, 57], [32, 819, 18, 22], [48, 653, 17, 140], [76, 604, 43, 105], [37, 730, 16, 18], [79, 661, 3, 71], [77, 209, 44, 62], [84, 673, 45, 95], [86, 648, 35, 77], [22, 535, 15, 84], [36, 815, 31, 103], [43, 244, 21, 62], [42, 62, 43, 94], [10, 558, 46, 71], [12, 755, 33, 3], [25, 534, 8, 144], [65, 518, 43, 82], [49, 310, 44, 31], [35, 409, 1, 17], [78, 643, 44, 12], [6, 423, 41, 45], [22, 747, 16, 102], [36, 837, 4, 84], [68, 445, 8, 41], [70, 765, 13, 78], [67, 488, 12, 86], [86, 532, 11, 104], [75, 666, 14, 47], [46, 818, 13, 66], [19, 551, 18, 105], [55, 460, 31, 99], [32, 492, 27, 37], [69, 680, 12, 27], [77, 818, 7, 7], [73, 461, 33, 131], [62, 177, 46, 134], [48, 396, 42, 79], [71, 760, 10, 95], [13, 653, 13, 1

# Resultado del Algoritmo Genetico

In [10]:
print(MB['Nombre'][241], MB['Precio'][241], '€')
print(MB['Nombre'][62], MB['Precio'][62], '€')
print(MB['Nombre'][76], MB['Precio'][76], '€')
print(MB['Nombre'][191], MB['Precio'][191], '€')


ASRock B550 Phantom Gaming 4 89.99 €
Gigabyte B650M D3HP 109.99 €
MSI PRO B760M-P DDR4 89.99 €
Gigabyte H510M K V2 86.13 €


In [394]:
Target = Poblacion[Fitness_Total.index(min(Fitness_Total))]
Fitness_optimizado = Fitness_Total[Fitness_Total.index(min(Fitness_Total))]
indice_optimizado = Fitness_Total.index(min(Fitness_Total))

if GPU['Chip'][Target[1]] == 'GeForce RTX 4090' or GPU['Chip'][Target[1]] == 'Radeon RX 7900 XTX':
    F_Alim = 62
else:
    F_Alim = 166

#Se compara el nombre de la cpu y se mira el zocalo, luego añadimos una placa base para ese zocalos
cpu_nombre = CPU['Nombre'][Target[0]]
if 'AMD' in cpu_nombre and ' 7' in cpu_nombre:
    Cpu_zocalo, Mb_ex = 'AM5', 62
elif 'AMD' in cpu_nombre and ' 7' not in cpu_nombre:
    Cpu_zocalo, Mb_ex = 'AM4', 241
elif '-10' in cpu_nombre or '-11' in cpu_nombre:
    Cpu_zocalo, Mb_ex = 'LGA1200', 191
elif '-12' in cpu_nombre or '-13' in cpu_nombre or '-14' in cpu_nombre:
    Cpu_zocalo, Mb_ex = 'LGA1700', 76

print('Cromosoma optimo:', Target)
print('Fitness del cromosoma:', Fitness_optimizado)

print('-'*50)
print('Procesador:', CPU['Nombre'][Target[0]], f'{CPU['Precio'][Target[0]]}€')
print('Targeta Grafica:', GPU['Fabricante'][Target[1]], GPU['Chip'][Target[1]], f'{GPU['Precio'][Target[1]]}€')
print('Memoria SSD', SSD['Nombre'][Target[2]], f'{SSD['Capacidad (GB)'][Target[2]]} GB', SSD['Interfaz'][Target[2]], f'{SSD['Precio'][Target[2]]}€')
print('Memoria RAM', RAM['Nombre'][Target[3]], f'{RAM['Capacidad'][Target[3]]} GB', f'{RAM['Precio'][Target[3]]}€')
print('-'*50)
print(f'Fuente de alimentación: {PS['Nombre'][F_Alim]}, {PS['Precio'][F_Alim]} €')
print(f'Caja: {CASE['Nombre'][4]}, {CASE['Precio'][4]} €')
print(f'Placa base: {MB['Nombre'][Mb_ex]}, {MB['Precio'][Mb_ex]} €')
print('-'*50)

print(f'Precio: {round(PrecioCrom(Poblacion)[indice_optimizado], 2) + PS['Precio'][F_Alim] + CASE['Precio'][0] + MB['Precio'][Mb_ex]}€ de {Stream_Lit_Pres}€ disponibles')

Cromosoma optimo: [42, 62, 43, 94]
Fitness del cromosoma: 241
--------------------------------------------------
Procesador: Intel Core i5-13400 228.99€
Targeta Grafica: Zotac GAMING Trinity OC GeForce RTX 4070 Ti 709.99€
Memoria SSD Samsung 980 250 GB M.2 PCIe 3.0 X4 59.99€
Memoria RAM Micron Technology 72ASS8G72LZ-2G3B2  64 GB 91€
--------------------------------------------------
Fuente de alimentación: Gigabyte UD750GM, 89.99 €
Caja: Corsair 4000D Airflow, 89.99 €
Placa base: Gigabyte Z790 AORUS ELITE AX, 229.99 €
--------------------------------------------------
Precio: 1499.94€ de 1500€ disponibles
